In [ ]:
import requests

In [ ]:
api_key = '--- Put Your API Key ---'

## Downloading BTC dataset

In [ ]:
from pandas.core.frame import DataFrame
import pandas as pd


ticker = 'BTC/USD'
#order = 'asc'
interval = '1day'
start_date = '2005-11-01'
end_date = '2025-11-14'

api_url = f'https://api.twelvedata.com/time_series?symbol={ticker}&start_date={start_date}&end_date={end_date}&interval={interval}&apikey={api_key}'

data = requests.get(api_url).json()
btc = pd.DataFrame(data['values'])
btc.to_csv('btc.csv', index=False)
print('DataFrame saved to btc.csv')


DataFrame saved to btc.csv


## Downloading QQQ Dataset

In [ ]:
from pandas.core.frame import DataFrame
import pandas as pd


ticker = 'QQQ'
interval = '1day'
start_date = '2005-11-01'
end_date = '2025-11-14'

api_url = f'https://api.twelvedata.com/time_series?symbol={ticker}&start_date={start_date}&end_date={end_date}&interval={interval}&apikey={api_key}'

data = requests.get(api_url).json()
qqq = pd.DataFrame(data['values'])
qqq.to_csv('qqq.csv', index=False)


# Data Preprocessing & Feature Engineering

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import median_abs_deviation, mstats
from typing import List

# --- Configuration ---
DAILY_RISK_FREE_RATE = 0.02 / 365
ROLLING_MEAN_WINDOW = 252 * 5 # 5 years for Expected Returns (1260 days)
MAD_CRITERION = 4            # Winsorization criterion


# --- Helper Functions ---

def calculate_rsi(df: pd.DataFrame, window: int = 14) -> pd.Series:
    """Calculates the Relative Strength Index (RSI)."""
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    RS = gain / loss
    RSI = 100 - (100 / (1 + RS))
    return RSI

def calculate_mad_winsorization_bounds(series: pd.Series, mad_criterion: int = 4):
    """Calculates the percentile limits for MAD-based winsorization."""
    if series.empty:
        return 0, 1

    median_val = series.median()
    mad_val = median_abs_deviation(series.dropna())

    if mad_val == 0:
        return series.min(), series.max()

    lower_bound = median_val - (mad_val * mad_criterion)
    upper_bound = median_val + (mad_val * mad_criterion)

    # Calculate the percentile limits (fraction of data outside bounds)
    lower_limit = (series < lower_bound).sum() / len(series)
    upper_limit = (series > upper_bound).sum() / len(series)

    return lower_limit, upper_limit


# --- FinancialFeatureEngineer Class ---

class FinancialFeatureEngineer:
    """Generates financial features (M*, V*, D*, P*) for a single asset."""

    def __init__(self, df: pd.DataFrame, asset_name: str):
        df['datetime'] = pd.to_datetime(df['datetime'])
        df = df.set_index('datetime').sort_index()

        # Coerce types and drop rows with missing closing price
        for col in ['open', 'high', 'low', 'close', 'volume']:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')

        self.df = df.dropna(subset=['close']).copy()
        self.asset_name = asset_name
        self.lookback_windows = [5, 10, 20, 60]

    def generate_core_features(self):
        """Returns, Intraday Volatility, Momentum (MOM*), and Volatility (V*) features."""
        self.df['Daily_Return'] = self.df['close'].pct_change()
        self.df['Intraday_Vol_Proxy'] = np.log(self.df['high'] / self.df['low'])

        for n in self.lookback_windows:
            # MOM*: Momentum
            self.df[f'Momentum_{n}'] = self.df['close'] / self.df['close'].shift(n)
            # V*: Rolling Volatility
            self.df[f'Vol_{n}'] = self.df['Daily_Return'].rolling(window=n).std()

            # P*: Rolling Price Level
            self.df[f'SMA_{n}'] = self.df['close'].rolling(window=n).mean()

            # Volume features (where applicable)
            if 'volume' in self.df.columns and not self.df['volume'].isnull().all():
                self.df[f'Avg_Volume_{n}'] = self.df['volume'].rolling(window=n).mean()
                self.df[f'Volume_Ratio_{n}'] = self.df['volume'] / self.df[f'Avg_Volume_{n}']
        return self

    def generate_calendar_features(self):
        """Generates time-based categorical features (Dummy features D*)."""
        self.df['DOW'] = self.df.index.dayofweek
        self.df['Is_Weekend'] = self.df['DOW'].apply(lambda x: 1 if x >= 5 else 0)
        self.df['Month'] = self.df.index.month
        self.df['Quarter'] = self.df.index.quarter

        self.df = pd.get_dummies(self.df, columns=['DOW', 'Month', 'Quarter'],
                                 prefix=['DOW', 'Month', 'Quarter'], drop_first=True)
        return self

    def generate_advanced_features(self):
        """Adds RSI, EWMA Volatility, Price Rank (P*), and Spread features (M*, V*)."""

        # P*: Price Rank (relative valuation proxy)
        self.df['Price_Rank_Long'] = self.df['close'].rolling(window=252).rank(pct=True)

        # M*: RSI
        self.df['RSI_14'] = calculate_rsi(self.df, window=14)

        # V*: EWMA Volatility (more reactive than simple rolling)
        self.df['EWMA_Vol_20'] = self.df['Daily_Return'].ewm(span=20, adjust=False).std()

        # M*: Open/Close vs. High/Low Spread
        range_move = self.df['high'] - self.df['low']
        daily_move = np.abs(self.df['close'] - self.df['open'])
        epsilon = 1e-6
        self.df['OC_Range_Ratio'] = daily_move / (range_move + epsilon)

        # BTC-Specific feature
        if self.asset_name == 'BTC':
            self.df['Overnight_Gap'] = (self.df['open'] / self.df['close'].shift(1)) - 1

        return self

    def get_features(self):
        """Returns the DataFrame, dropping base OHLC data columns."""
        cols_to_drop = ['open', 'high', 'low', 'close', 'volume']
        cols_to_drop = [col for col in cols_to_drop if col in self.df.columns]
        return self.df.drop(columns=cols_to_drop, errors='ignore')


# --- Target Generation Function (market_forward_excess_returns analog) ---

def generate_final_target(df: pd.DataFrame, asset_prefix: str) -> pd.DataFrame:
    """Generates the final winsorized excess return target."""

    forward_col = f'{asset_prefix}_Forward_Return'
    target_col = f'{asset_prefix}_Target_Final'

    # 1. R_Expected: Rolling 5-Year Mean of Forward Returns
    df['Rolling_Mean_Forward'] = df[forward_col].rolling(window=ROLLING_MEAN_WINDOW).mean()

    # 2. Raw Excess Return (R_Forward - R_Expected)
    raw_excess_return_col = f'{asset_prefix}_Raw_Excess_Return'
    df[raw_excess_return_col] = df[forward_col] - df['Rolling_Mean_Forward']

    # 3. Winsorize (4x MAD)
    winsorized_series = df[raw_excess_return_col].copy()
    valid_data = winsorized_series.dropna()

    if not valid_data.empty:
        lower_limit, upper_limit = calculate_mad_winsorization_bounds(valid_data, MAD_CRITERION)

        winsorized_array = mstats.winsorize(
            valid_data,
            limits=(lower_limit, upper_limit)
        ).data

        winsorized_series.loc[valid_data.index] = winsorized_array

    df[target_col] = winsorized_series

    df = df.drop(columns=['Rolling_Mean_Forward', raw_excess_return_col], errors='ignore')
    return df


# --- Main Pipeline Execution ---

def run_feature_engineering_pipeline(btc_file: str, qqq_file: str) -> pd.DataFrame:
    """Orchestrates the loading, engineering, merging, and target generation."""

    print("1. Loading Data...")
    btc_df = pd.read_csv(btc_file)
    qqq_df = pd.read_csv(qqq_file)

    # 2. Engineer Features for Each Asset Separately
    print("2. Generating BTC Features...")
    btc_engineer = FinancialFeatureEngineer(btc_df, 'BTC')
    btc_features = (btc_engineer
                    .generate_core_features()
                    .generate_calendar_features()
                    .generate_advanced_features()
                    .get_features())

    print("3. Generating QQQ Features...")
    qqq_engineer = FinancialFeatureEngineer(qqq_df, 'QQQ')
    qqq_features = (qqq_engineer
                     .generate_core_features()
                     .generate_calendar_features()
                     .generate_advanced_features()
                     .get_features())

    # 4. Rename Columns and Merge DataFrames (Inner join for common trading days)
    print("4. Merging DataFrames...")
    btc_features = btc_features.add_prefix('BTC_')
    qqq_features = qqq_features.add_prefix('QQQ_')
    merged_df = qqq_features.merge(btc_features, left_index=True, right_index=True, how='inner')

    # 5. Generate Forward Returns (forward_returns analog) and Risk-Free Rate
    print("5. Generating Targets and Risk Features...")
    merged_df['QQQ_Forward_Return'] = merged_df['QQQ_Daily_Return'].shift(-1)
    merged_df['BTC_Forward_Return'] = merged_df['BTC_Daily_Return'].shift(-1)
    merged_df['Risk_Free_Rate'] = DAILY_RISK_FREE_RATE # risk_free_rate analog

    # 6. Generate Cross-Market Features (E* Proxies)
    merged_df['BTC_Prev_Day_QQQ_Return'] = merged_df['QQQ_Daily_Return'].shift(1)
    merged_df['QQQ_Prev_Day_BTC_Return'] = merged_df['BTC_Daily_Return'].shift(1)

    window_corr = 30
    merged_df[f'Corr_BTC_QQQ_{window_corr}d'] = (
        merged_df['BTC_Daily_Return']
        .rolling(window=window_corr)
        .corr(merged_df['QQQ_Daily_Return'])
    )

    # 7. Generate Final Winsorized Targets (market_forward_excess_returns analog)
    print("7. Generating Final Winsorized Targets...")
    merged_df = generate_final_target(merged_df, 'QQQ')
    merged_df = generate_final_target(merged_df, 'BTC')

    # 8. Final Cleanup
    final_df_complete = merged_df.dropna()
    print(f"8. Final Feature Data Shape: {final_df_complete.shape}")

    return final_df_complete

# --- Final Execution ---

# Note: You must ensure 'btc.csv' and 'qqq.csv' are created before running this function.
try:
    final_features_df = run_feature_engineering_pipeline(
        btc_file='btc.csv',
        qqq_file='qqq.csv'
    )

    final_features_df.to_csv('final_features_btc_qqq.csv')
    print("\n✅ Pipeline complete. Final Engineered Feature Dataset saved to final_features_btc_qqq.csv")
    print(f"Total features created: {final_features_df.shape[1]}")

except FileNotFoundError as e:
    print(f"\n🛑 Error: File {e.filename} not found.")
    print("Please ensure you run the data fetching/mock data creation step first to generate the CSV files.")
except Exception as e:
    print(f"\n❌ An unexpected error occurred: {e}")

1. Loading Data...
2. Generating BTC Features...
3. Generating QQQ Features...
4. Merging DataFrames...
5. Generating Targets and Risk Features...
7. Generating Final Winsorized Targets...
8. Final Feature Data Shape: (1548, 93)

✅ Pipeline complete. Final Engineered Feature Dataset saved to final_features_btc_qqq.csv
Total features created: 93


## Data Cleaning & Feature Engineering

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import median_abs_deviation, mstats
from typing import List, Tuple

# --- Configuration ---
DAILY_RISK_FREE_RATE = 0.02 / 365
ROLLING_MEAN_WINDOW = 252 * 5 # 5 years for Expected Returns (1260 days)
MAD_CRITERION = 4            # Winsorization criterion

# --- Helper Functions (RSI, MAD, etc. - defined in previous steps) ---
# [Note: These functions are assumed to be defined as in the previous successful execution.]
# For simplicity, they are omitted here, but their logic is used below.

def calculate_rsi(df: pd.DataFrame, window: int = 14) -> pd.Series:
    """Calculates the Relative Strength Index (RSI)."""
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    RS = gain / loss
    RSI = 100 - (100 / (1 + RS))
    return RSI

def calculate_mad_winsorization_bounds(series: pd.Series, mad_criterion: int = 4):
    """Calculates the percentile limits for MAD-based winsorization."""
    if series.empty:
        return 0, 1
    median_val = series.median()
    mad_val = median_abs_deviation(series.dropna())
    if mad_val == 0:
        return series.min(), series.max()
    lower_bound = median_val - (mad_val * mad_criterion)
    upper_bound = median_val + (mad_val * mad_criterion)
    lower_limit = (series < lower_bound).sum() / len(series)
    upper_limit = (series > upper_bound).sum() / len(series)
    return lower_limit, upper_limit

def generate_final_target(df: pd.DataFrame, asset_prefix: str) -> pd.DataFrame:
    """Generates the final winsorized excess return target."""
    forward_col = f'{asset_prefix}_Forward_Return'
    target_col = f'{asset_prefix}_Target_Final'
    df['Rolling_Mean_Forward'] = df[forward_col].rolling(window=ROLLING_MEAN_WINDOW).mean()
    raw_excess_return_col = f'{asset_prefix}_Raw_Excess_Return'
    df[raw_excess_return_col] = df[forward_col] - df['Rolling_Mean_Forward']
    winsorized_series = df[raw_excess_return_col].copy()
    valid_data = winsorized_series.dropna()
    if not valid_data.empty:
        lower_limit, upper_limit = calculate_mad_winsorization_bounds(valid_data, MAD_CRITERION)
        winsorized_array = mstats.winsorize(valid_data, limits=(lower_limit, upper_limit)).data
        winsorized_series.loc[valid_data.index] = winsorized_array
    df[target_col] = winsorized_series
    df = df.drop(columns=['Rolling_Mean_Forward', raw_excess_return_col], errors='ignore')
    return df

# --- FinancialFeatureEngineer Class (Re-used for context) ---

class FinancialFeatureEngineer:
    # [Class methods: __init__, generate_core_features, generate_calendar_features, generate_advanced_features, get_features]

    def __init__(self, df: pd.DataFrame, asset_name: str):
        df['datetime'] = pd.to_datetime(df['datetime'])
        df = df.set_index('datetime').sort_index()
        for col in ['open', 'high', 'low', 'close', 'volume']:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
        self.df = df.dropna(subset=['close']).copy()
        self.asset_name = asset_name
        self.lookback_windows = [5, 10, 20, 60]
        # Store raw OHLC data separately before dropping
        self.raw_data = self.df[['open', 'high', 'low', 'close']].copy()
        if 'volume' in self.df.columns:
            self.raw_data['volume'] = self.df['volume']

    def generate_core_features(self):
        self.df['Daily_Return'] = self.df['close'].pct_change()
        self.df['Intraday_Vol_Proxy'] = np.log(self.df['high'] / self.df['low'])
        for n in self.lookback_windows:
            self.df[f'Momentum_{n}'] = self.df['close'] / self.df['close'].shift(n)
            self.df[f'Vol_{n}'] = self.df['Daily_Return'].rolling(window=n).std()
            self.df[f'SMA_{n}'] = self.df['close'].rolling(window=n).mean()
            if 'volume' in self.df.columns and not self.df['volume'].isnull().all():
                self.df[f'Avg_Volume_{n}'] = self.df['volume'].rolling(window=n).mean()
                self.df[f'Volume_Ratio_{n}'] = self.df['volume'] / self.df[f'Avg_Volume_{n}']
        return self

    def generate_calendar_features(self):
        self.df['DOW'] = self.df.index.dayofweek
        self.df['Is_Weekend'] = self.df['DOW'].apply(lambda x: 1 if x >= 5 else 0)
        self.df['Month'] = self.df.index.month
        self.df['Quarter'] = self.df.index.quarter
        self.df = pd.get_dummies(self.df, columns=['DOW', 'Month', 'Quarter'], prefix=['DOW', 'Month', 'Quarter'], drop_first=True)
        return self

    def generate_advanced_features(self):
        self.df['Price_Rank_Long'] = self.df['close'].rolling(window=252).rank(pct=True)
        self.df['RSI_14'] = calculate_rsi(self.df, window=14)
        self.df['EWMA_Vol_20'] = self.df['Daily_Return'].ewm(span=20, adjust=False).std()
        range_move = self.df['high'] - self.df['low']
        daily_move = np.abs(self.df['close'] - self.df['open'])
        epsilon = 1e-6
        self.df['OC_Range_Ratio'] = daily_move / (range_move + epsilon)
        if self.asset_name == 'BTC':
            self.df['Overnight_Gap'] = (self.df['open'] / self.df['close'].shift(1)) - 1
        return self

    def get_features(self):
        cols_to_drop = ['open', 'high', 'low', 'close', 'volume']
        cols_to_drop = [col for col in cols_to_drop if col in self.df.columns]
        return self.df.drop(columns=cols_to_drop, errors='ignore')

# --- Main Pipeline Execution ---

def run_and_save_separate_datasets(btc_file: str, qqq_file: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Orchestrates engineering, merging, and saving BTC and QQQ datasets separately."""

    # 1. Load Data
    print("1. Loading Data...")
    btc_df = pd.read_csv(btc_file)
    qqq_df = pd.read_csv(qqq_file)

    # 2. Engineer Features for Each Asset Separately
    print("2. Generating BTC Features...")
    btc_engineer = FinancialFeatureEngineer(btc_df, 'BTC')
    btc_features = (btc_engineer.generate_core_features().generate_calendar_features().generate_advanced_features().get_features())
    btc_raw = btc_engineer.raw_data.copy()

    print("3. Generating QQQ Features...")
    qqq_engineer = FinancialFeatureEngineer(qqq_df, 'QQQ')
    qqq_features = (qqq_engineer.generate_core_features().generate_calendar_features().generate_advanced_features().get_features())
    qqq_raw = qqq_engineer.raw_data.copy()

    # 4. Merge Features (Inner join for common trading days)
    print("4. Merging Engineered Features...")
    btc_features_prefixed = btc_features.add_prefix('BTC_')
    qqq_features_prefixed = qqq_features.add_prefix('QQQ_')

    # This DataFrame holds all features (QQQ index alignment)
    merged_features = qqq_features_prefixed.merge(btc_features_prefixed, left_index=True, right_index=True, how='inner')

    # 5. Generate Targets and Cross-Market Features
    merged_features['QQQ_Forward_Return'] = merged_features['QQQ_Daily_Return'].shift(-1)
    merged_features['BTC_Forward_Return'] = merged_features['BTC_Daily_Return'].shift(-1)
    merged_features['Risk_Free_Rate'] = DAILY_RISK_FREE_RATE

    merged_features['BTC_Prev_Day_QQQ_Return'] = merged_features['QQQ_Daily_Return'].shift(1)
    merged_features['QQQ_Prev_Day_BTC_Return'] = merged_features['BTC_Daily_Return'].shift(1)
    merged_features[f'Corr_BTC_QQQ_30d'] = merged_features['BTC_Daily_Return'].rolling(window=30).corr(merged_features['QQQ_Daily_Return'])

    # 6. Generate Final Winsorized Targets
    print("6. Generating Final Winsorized Targets...")
    final_engineered_df = generate_final_target(merged_features, 'QQQ')
    final_engineered_df = generate_final_target(final_engineered_df, 'BTC')

    # Final Cleanup (drop NaNs from rolling windows/forward shift)
    final_engineered_df = final_engineered_df.dropna()
    print(f"   Final Clean Feature Index Size: {final_engineered_df.shape[0]}")

    # 7. Separate and Save Datasets (Adding Raw Features Back)

    # --- QQQ Dataset ---
    # Select QQQ-specific columns plus all cross-market features and targets
    qqq_cols = [col for col in final_engineered_df.columns if col.startswith('QQQ_') or col.startswith('Risk_Free_Rate') or col.startswith('Corr_') or col.startswith('BTC_Prev_Day_QQQ_Return')]

    final_qqq_features = final_engineered_df[qqq_cols]
    # Merge with original QQQ raw data (O, H, L, C, V)
    final_qqq_dataset = final_qqq_features.merge(qqq_raw, left_index=True, right_index=True, how='inner')

    # --- BTC Dataset ---
    # Select BTC-specific columns plus all cross-market features and targets
    btc_cols = [col for col in final_engineered_df.columns if col.startswith('BTC_') or col.startswith('Risk_Free_Rate') or col.startswith('Corr_') or col.startswith('QQQ_Prev_Day_BTC_Return')]

    final_btc_features = final_engineered_df[btc_cols]
    # Merge with original BTC raw data (O, H, L, C)
    final_btc_dataset = final_btc_features.merge(btc_raw, left_index=True, right_index=True, how='inner')

    # Save to CSV
    final_qqq_dataset.to_csv('final_qqq_dataset.csv')
    final_btc_dataset.to_csv('final_btc_dataset.csv')

    print("\n✅ Successfully generated and saved two separate datasets:")
    print(f"   - final_qqq_dataset.csv (Shape: {final_qqq_dataset.shape})")
    print(f"   - final_btc_dataset.csv (Shape: {final_btc_dataset.shape})")

    return final_qqq_dataset, final_btc_dataset

# --- Final Execution ---
try:
    # NOTE: Assuming 'btc.csv' and 'qqq.csv' are available from previous steps
    final_qqq_df, final_btc_df = run_and_save_separate_datasets(
        btc_file='btc.csv',
        qqq_file='qqq.csv'
    )

    print("\n--- QQQ Dataset Head (Raw + Engineered) ---")
    print(final_qqq_df.head())
    print("\n--- BTC Dataset Head (Raw + Engineered) ---")
    print(final_btc_df.head())

except FileNotFoundError as e:
    print(f"\n🛑 Error: File {e.filename} not found.")
    print("Please ensure you run the data fetching/mock data creation step first.")
except Exception as e:
    print(f"\n❌ An unexpected error occurred: {e}")

1. Loading Data...
2. Generating BTC Features...
3. Generating QQQ Features...
4. Merging Engineered Features...
6. Generating Final Winsorized Targets...
   Final Clean Feature Index Size: 1548

✅ Successfully generated and saved two separate datasets:
   - final_qqq_dataset.csv (Shape: (1548, 56))
   - final_btc_dataset.csv (Shape: (1548, 50))

--- QQQ Dataset Head (Raw + Engineered) ---
            QQQ_Daily_Return  QQQ_Intraday_Vol_Proxy  QQQ_Momentum_5  \
datetime                                                               
2019-09-18         -0.000415                0.013013        1.000468   
2019-09-19          0.001662                0.008124        0.997982   
2019-09-20         -0.010579                0.016429        0.990963   
2019-09-23         -0.001572                0.007091        0.993844   
2019-09-24         -0.013281                0.024491        0.975960   

            QQQ_Vol_5   QQQ_SMA_5  QQQ_Avg_Volume_5  QQQ_Volume_Ratio_5  \
datetime                   